In [8]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import numpy as np
from shapely.geometry import LineString
from shapely.ops import split
import math
from shapely.geometry import Point
from shapely.strtree import STRtree
import pandas as pd
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union
import shapely
from shapely.geometry import Point
import networkx as nx
import random
import time
import openpyxl
import pandas as pd

def classify_geometry(geom):
    if geom.is_empty:
        return 'EmptyGeometry'
    elif geom.type == 'Point':
        return 'Point'
    elif geom.type == 'LineString':
        return 'LineString'
    elif geom.type == 'Polygon':
        # 区分单部件和多部件多边形
        if geom.is_empty:
            return 'EmptyPolygon'
        elif geom.num_parts == 1:
            return 'SinglePartPolygon'
        else:
            return 'MultiPartPolygon'
    elif geom.type == 'MultiPoint':
        return 'MultiPoint'
    elif geom.type == 'MultiLineString':
        return 'MultiLineString'
    elif geom.type == 'MultiPolygon':
        return 'MultiPolygon'
    else:
        return 'Other'

def SHDI(s):   #香农多样性指数
    ans = 0
    min_p = 1e-10
    for si in s:
        ans += si * math.log(max(si, min_p))
    return -ans

def angle_cal(start_point, end_point):
    delta_x = end_point[0] - start_point[0]
    delta_y = end_point[1] - start_point[1]
    angle = math.degrees(math.atan2(delta_y, delta_x))
    # 调整角度使其在0-360之间
    angle = angle if angle >= 0 else 360 + angle
    return angle

def firstOrder_feature_cal(linestring, firstOrder_feature):
    last_last_angle = 0
    last_angle = 0
    for i in range(len(linestring.coords) - 1):
        # 获取当前段的两端点
        p1 = linestring.coords[i]
        p2 = linestring.coords[i + 1]
        # 计算长度
        length = LineString([p1, p2]).length
        # 计算方向（角度），这里假设返回的角度是相对于正北方向
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        angle = math.degrees(math.atan2(dy, dx))  # atan2返回的是弧度
        # 调整角度使其在0-360之间
        angle = angle if angle >= 0 else 360 + angle
        firstOrder_feature[int(angle/30)] +=  length
        if i > 0 :
            if abs(last_angle -angle)  < 45:
                firstOrder_feature[12] += 1
        last_last_angle = last_angle
        last_angle = angle
    return firstOrder_feature

excel_areas = ['常州','杭州', '湖州', '嘉兴', '上海','苏州','无锡','镇江']
all_features_list = []
for area in excel_areas:
    time1 = time.time()

    
    # 设定道路的目录路径
    directory_path ='C:/Users/44156/Desktop/科研项目/空间基因/打断且带序号的中心线/' + area
    # 列出目录下所有的文件
    file_list = os.listdir(directory_path)
    # 筛选出所有的.shp文件
    shp_files = [f for f in file_list if f.endswith('.shp')]
    # 检查是否只有一个.shp文件
    if len(shp_files) == 1:
        # 获取唯一的.shp文件名
        shp_file = shp_files[0]
        # 读取.shp文件
        gdf = gpd.read_file(directory_path + '/' + shp_file)
    else:
        print("没有找到唯一的.shp文件，或者找到了多个.shp文件。跳过该地区：" + area)  
        break
        
    # 设定村落的目录路径
    directory_path ='C:/Users/44156/Desktop/科研项目/空间基因/data/' + area +'村落'
    # 列出目录下所有的文件
    file_list = os.listdir(directory_path)
    # 筛选出所有的.shp文件
    shp_files = [f for f in file_list if f.endswith('.shp')]
    # 检查是否只有一个.shp文件
    if len(shp_files) == 1:
        # 获取唯一的.shp文件名
        shp_file = shp_files[0]
        # 读取.shp文件
        gdf_cunluo = gpd.read_file(directory_path + '/' + shp_file)
    else:
        print("没有找到唯一的.shp文件，或者找到了多个.shp文件。跳过该地区：" + area)  
        break

    # 设定用地的目录路径
    directory_path ='C:/Users/44156/Desktop/科研项目/空间基因/data/' + area +'用地'
    # 列出目录下所有的文件
    file_list = os.listdir(directory_path)
    # 筛选出所有的.shp文件
    shp_files = [f for f in file_list if f.endswith('.shp')]
    # 检查是否只有一个.shp文件
    if len(shp_files) == 1:
        # 获取唯一的.shp文件名
        shp_file = shp_files[0]
        # 读取.shp文件
        gdf_useland = gpd.read_file(directory_path + '/' + shp_file)
    else:
        print("没有找到唯一的.shp文件，或者找到了多个.shp文件。跳过该地区：" + area)  
        break
        
    gdf_useland = gdf_useland[gdf_useland["CLASS_NAME"] == "建设用地"]
    gdf_useland = gdf_useland.to_crs(gdf.crs)
    gdf_cunluo = gdf_cunluo.to_crs(gdf.crs)
    unique_values = gdf_cunluo['cmbh'].unique()
    all_features = np.zeros((0, 10))
    pointer = -1
    
    for cmbh in unique_values:
        # 开始时间
        time2 = time.time() 
        gdf2_cunluo = gdf_cunluo[gdf_cunluo['cmbh'] == cmbh]
        if not gdf2_cunluo.geometry.is_valid.all(): continue
        intersection_mask = gdf.intersects(gdf2_cunluo.geometry.unary_union)
        # 使用布尔序列筛选出相交的线
        gdf2 = gdf[intersection_mask]  
        if len(gdf2) == 0: continue

        start_points = []                                                             # 这些可以看作是从行数到其它变量的字典
        end_points = []
        start_angles = []
        end_angles = []
        road_index = []
        secondOrder_feature = np.zeros(3)  
        
        for index, row in gdf2.iterrows():
            start_points.append(Point(row.geometry.coords[0]))
            end_points.append(Point(row.geometry.coords[-1]))
            start_angle = angle_cal(row.geometry.coords[0], row.geometry.coords[1])
            end_angle = angle_cal(row.geometry.coords[-2], row.geometry.coords[-1])
            start_angles.append(start_angle)
            end_angles.append(end_angle)
            road_index.append(row['index'])

        tree1 = STRtree(start_points)
        tree2 = STRtree(end_points)
        for i in range(len(road_index)):
            start_point = start_points[i]
            near_index1 = tree1.query(start_point.buffer(0.01))
            near_index2 = tree2.query(start_point.buffer(0.01))
            end_point = end_points[i]
            near_index3 = tree1.query(end_point.buffer(0.01))
            near_index4 = tree2.query(end_point.buffer(0.01))
            if len(near_index1) + len(near_index2) == 1 and len(near_index3) + len(near_index4) == 1:
                gdf2 = gdf2.drop(road_index[i])
        if len(gdf2) == 0: 
            print("删去了", area, cmbh)
            continue

        gdf2_buffer = gdf2.buffer(1)                                                           #这部分是在准备闭环检测的数据
        gdf2_buffer_reverse = gdf2_cunluo.difference(gdf2_buffer.unary_union)
        filtered_gdf2_buffer_reverse = gdf2_buffer_reverse
        if gdf2_buffer_reverse.iloc[0].geom_type.startswith('Multi'):
            filtered_gdf2_buffer_reverse = [poly for poly in gdf2_buffer_reverse.iloc[0].geoms if poly.area >= gdf2_cunluo.iloc[0].geometry.area * 0.002]
        all_features = np.vstack((all_features, np.zeros((1, 10))))
        pointer += 1


        Rid_to_Spoint = {}                                                                     #准备找到道路和建设用地的相交点
        gdf2_useland = gdf_useland[gdf_useland["cmbh"] == cmbh]
        boundary_union = unary_union(gdf2_useland.boundary)
        all_Spoints = []
        
        for index, row in gdf2.iterrows():  
            Spoints = []
            # 计算多边形边界与线的交点
            intersection_points = boundary_union.intersection(row.geometry)
            # 检查交点类型，如果是MultiPoint，则交点有多个
            if isinstance(intersection_points, shapely.geometry.multipoint.MultiPoint):
                # 交点为多个点
                Spoints = list(intersection_points.geoms)[0:1]
            elif isinstance(intersection_points, shapely.geometry.Point):
                # 交点为一个点
                Spoints = [intersection_points]
            Rid_to_Spoint[index] = Spoints
            test_gdf = gpd.GeoDataFrame(geometry = Spoints, crs = gdf.crs)
            if len(Spoints) > 0:
                all_Spoints +=  Spoints
        if len(all_Spoints) > 20:
            # 如果元素数量大于20，使用random.sample均匀抽取20个元素
            all_Spoints = random.sample(all_Spoints, 20)
#        test_gdf = gpd.GeoDataFrame(geometry=all_Spoints, crs = gdf.crs)
#        # 创建一个绘图和轴对象
#        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
#        gdf2_useland.plot(ax=ax, color='red')
#        gdf2.plot(ax=ax, color='green')
#        test_gdf.plot(ax=ax, color='blue', markersize=20) 
            


        
        G = nx.Graph()                                                        #准备图结构
        # 用于存储节点坐标的字典
        node_coords = {}                                                      #从点坐标到图节点id
        node_amounts = {}                                                     #从图节点id到点的重复次数
        
        # 遍历线段并添加到图中
        for index, row in gdf2.iterrows():
            # 假设 gdf 的几何列名为 'geometry'
            line = row['geometry']
            # 获取线段的起点和终点坐标
            start_coord = line.coords[0]
            end_coord = line.coords[-1]
            
            # 添加节点，并确保每个坐标只添加一次
            if start_coord not in node_coords:
                node_id = index + 0.1
                node_coords[start_coord] = node_id
                node_amounts[node_id] = 1 
                G.add_node(node_id, pos=start_coord)
            else :node_amounts[node_coords[start_coord]] += 1
            if end_coord not in node_coords:
                node_id = index + 0.2
                node_coords[end_coord] = node_id
                node_amounts[node_id] = 1 
                G.add_node(node_id, pos=end_coord)
            else :node_amounts[node_coords[end_coord]] += 1
            # 添加边，并计算权重（这里使用线段的长度）
            G.add_edge(node_coords[start_coord], node_coords[end_coord], weight=line.length)
            #添加兴趣点
            for Spoint in Rid_to_Spoint[index]:
                G.add_node(Spoint, pos=Spoint)
                G.add_edge(node_coords[start_coord], Spoint, weight=Spoint.distance(Point(*start_coord)))
                G.add_edge(node_coords[end_coord], Spoint, weight=Spoint.distance(Point(*end_coord)))

        
        


        
        firstOrder_feature = np.zeros(13)
        for index, row in gdf2.iterrows():
            linestring = row.geometry
            if linestring.geom_type.startswith('Multi'):
                # 处理多部分几何对象
                # 例如，分解为单部分几何对象
                for part in linestring.geoms:
                    firstOrder_feature = firstOrder_feature_cal(part, firstOrder_feature)               
            else:
                firstOrder_feature = firstOrder_feature_cal(linestring, firstOrder_feature)
#         print(np.round(firstOrder_feature, 3))
        all_features[pointer,0] = cmbh                                                                  #编号，基本属性
        all_features[pointer,1] = SHDI(firstOrder_feature[0:12] / firstOrder_feature[0:12].sum())
        all_features[pointer,2] = firstOrder_feature[12] / len(gdf2)
        all_features[pointer,3] = gdf2.length.sum()
         


        
        for index, start_point in enumerate(start_points):
            if start_point == 0: continue
            near_index1 = tree1.query(start_point.buffer(3))
            near_index2 = tree2.query(start_point.buffer(3))
            point_feature = []
            for near_index in near_index1:
                point_feature = point_feature + [start_angles[near_index]]
                start_points[near_index] = 0
            for near_index in near_index2:
                point_feature = point_feature + [end_angles[near_index]]
                end_points[near_index] = 0
#            if cmbh == 249: print(len(point_feature), road_index[index])
            if len(point_feature) == 1: 
                secondOrder_feature[0] += 1
#                if end_points[index] == 0: secondOrder_feature[0] += 1
#                elif len(tree1.query(end_points[index].buffer(3))) + len(tree2.query(end_points[index].buffer(3))) == 1: end_points[index] = 0
#                else: secondOrder_feature[0] += 1
            elif len(point_feature) == 3: secondOrder_feature[1] += 1
            elif len(point_feature) == 6: secondOrder_feature[2] += 1
#             print(point_feature, road_index[index])
        
        for index, end_point in enumerate(end_points):
            if end_point == 0: continue
            near_index1 = tree1.query(end_point.buffer(3))
            near_index2 = tree2.query(end_point.buffer(3))
            point_feature = []
            for near_index in near_index1:
                point_feature = point_feature + [start_angles[near_index]]
                start_points[near_index] = 0
            for near_index in near_index2:
                point_feature = point_feature + [end_angles[near_index]]
                end_points[near_index] = 0
#            if cmbh == 249: print(len(point_feature), road_index[index])
            if len(point_feature) == 1: secondOrder_feature[0] += 1
            elif len(point_feature) == 3: secondOrder_feature[1] += 1
            elif len(point_feature) == 6: secondOrder_feature[2] += 1
#             print(point_feature, road_index[index])  
        all_features[pointer,4:7] = secondOrder_feature / max(1, secondOrder_feature.sum())        #空间结构指标          
        all_features[pointer,7] = len(filtered_gdf2_buffer_reverse)

#        print(cmbh)
#        if cmbh == 252:
#            test_gdf = gpd.GeoDataFrame(geometry=all_Spoints, crs = gdf.crs)
#            # 创建一个绘图和轴对象
#            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
#            gdf2_useland.plot(ax=ax, color='red')
#            gdf2.plot(ax=ax, color='green')
#            test_gdf.plot(ax=ax, color='blue', markersize=20) 

        global_reaching_centrality = 0
        global_efficiency = 0
        path_mounts = 0
        if len(all_Spoints) > 2:
            for i in range(len(all_Spoints) - 1):
                for j in range(i+1, len(all_Spoints)):
                    source = all_Spoints[i]
                    target = all_Spoints[j]
                    if source == target:continue
                    if nx.has_path(G, source, target):
                        # 计算路径长度
                        shortest_path_length = nx.shortest_path_length(G, source=source, target=target, weight='weight')
                        global_reaching_centrality += shortest_path_length
                        global_efficiency += (1 / shortest_path_length)
                        path_mounts += 1
            if path_mounts > 0: 
                global_reaching_centrality = global_reaching_centrality / path_mounts
                global_efficiency = global_efficiency / path_mounts
        all_features[pointer,8] = global_reaching_centrality
        all_features[pointer,9] = global_efficiency                                                  #图论指标

        
        time3 = time.time()
        
        print(area, cmbh, 'over', '耗时为', time3 - time2)
        print('******************************************************************************')
    time4 = time.time()
    print(area, 'over', '耗时为', time4 - time1)
    all_features_list.append(all_features)
    column_names = ['cmbh', '方向熵', '平均曲率', '道路总长度', '尽端率', 'T型率', 'X型率',  '道路环度', '全局可达中心率', '全局效率']
    with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        output_indicators = pd.DataFrame(all_features, columns=column_names)
        output_indicators.to_excel(writer, sheet_name=area, index=False)

常州 5 over 耗时为 0.7100372314453125
******************************************************************************
常州 249 over 耗时为 0.27813720703125
******************************************************************************
常州 252 over 耗时为 0.4216630458831787
******************************************************************************
常州 255 over 耗时为 1.4447340965270996
******************************************************************************
常州 257 over 耗时为 0.3845338821411133
******************************************************************************
常州 344 over 耗时为 0.32827210426330566
******************************************************************************
常州 364 over 耗时为 0.4978458881378174
******************************************************************************
常州 369 over 耗时为 0.08178329467773438
******************************************************************************
常州 392 over 耗时为 0.3042914867401123
*******************************************************

In [ ]:
from sklearn.cluster import KMeans

data = np.vstack(all_features_list)
cmbhs = data[:,0]
data = data[:,1:]

# data = all_features.copy()
col_min = data.min(axis=0)
col_max = data.max(axis=0)
# 归一化
data = (data - col_min) / (col_max - col_min)

# 初始化KMeans聚类器
kmeans = KMeans(n_clusters=5)

# 拟合数据
kmeans.fit(data)

data = np.hstack(cmbhs, data)
  
# 获取聚类中心
centers = kmeans.cluster_centers_

# 获取每个数据点的聚类标签
labels = kmeans.labels_
pointer = -1
for i, area in enumerate(excel_areas):
    # 设定道路的目录路径
    directory_path ='C:/Users/44156/Desktop/科研项目/空间基因/打断且带序号的中心线/' + area
    # 列出目录下所有的文件
    file_list = os.listdir(directory_path)
    # 筛选出所有的.shp文件
    shp_files = [f for f in file_list if f.endswith('.shp')]
    # 检查是否只有一个.shp文件
    if len(shp_files) == 1:
        # 获取唯一的.shp文件名
        shp_file = shp_files[0]
        # 读取.shp文件
        gdf = gpd.read_file(directory_path + '/' + shp_file)
    else:
        print("没有找到唯一的.shp文件，或者找到了多个.shp文件。跳过该地区：" + area)  
        break
    # 设定村落的目录路径
    directory_path ='C:/Users/44156/Desktop/科研项目/空间基因/data/' + area +'村落'
    # 列出目录下所有的文件
    file_list = os.listdir(directory_path)
    # 筛选出所有的.shp文件
    shp_files = [f for f in file_list if f.endswith('.shp')]
    # 检查是否只有一个.shp文件
    if len(shp_files) == 1:
        # 获取唯一的.shp文件名
        shp_file = shp_files[0]
        # 读取.shp文件
        gdf_cunluo = gpd.read_file(directory_path + '/' + shp_file)
    else:
        print("没有找到唯一的.shp文件，或者找到了多个.shp文件。跳过该地区：" + area)  

    gdf_cunluo['label'] = None
    for i in  range(len(all_features_list[i])):
        pointer += 1
        gdf_cunluo.loc[gdf_cunluo['cmbh'] == cmbhs[pointer], 'label'] = int(labels[pointer])
    gdf_cunluo.to_file('C:/Users/44156/Desktop/科研项目/空间基因/太湖流域的道路分类结果（shp）/' + area + '.shp', encoding='utf-8', driver='ESRI Shapefile')